In [5]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import random



In [10]:

class RdmDataset(Dataset):
    def __init__(self, root_dir, train=True):
        self.data = []
        self.labels = []
        classes = {'GOUP': 0, 'BLNC': 1, 'DOWN': 2}
        
        folders = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        split_idx = int(len(folders) * 0.8)  # 80-20 split for train-test
        selected_folders = folders[:split_idx] if train else folders[split_idx:]
        print(f"Is_train? {train}")
        print(selected_folders)
        
        for folder in selected_folders:
            folder_path = os.path.join(root_dir, folder)
            for file in os.listdir(folder_path):
                if file.endswith('.npy'):
                    class_label = [classes[key] for key in classes if key in file.upper()][0]
                    rdm_data = np.load(os.path.join(folder_path, file))
                    
                    if rdm_data.shape[1] != 23 or rdm_data.shape[2] != 13:
                        print(rdm_data.shape)
                        break
                    
                    self.data.append(torch.tensor(rdm_data, dtype=torch.float).unsqueeze(0))  # Add channel dimension
                    self.labels.append(class_label)
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Assume all frames within a sequence are of the same shape, which is a prerequisite.
        # If they are not, you need to resize them or handle it as needed.
        rdm_sequence = self.data[idx]
        label = self.labels[idx]
        
        # If resizing is necessary (e.g., if frames in the same sequence could have different shapes),
        # you can use torchvision.transforms.Resize:
        # resize_transform = Resize(size=(desired_height, desired_width))
        # rdm_sequence = torch.stack([resize_transform(frame) for frame in rdm_sequence])

        # Padding the sequence to the max length will be done in the collate_fn
    
        return rdm_sequence, self.labels[idx]
    
def collate_fn(batch):
    # Separate the sequences and labels
    sequences, labels = zip(*batch)
    
    # Remove the extra leading dimension and calculate lengths for each sequence in the batch
    # The extra dimension is removed by using .squeeze(0), which removes the dimension of size 1 at position 0
    sequences = [seq.squeeze(0) for seq in sequences]  # Adjusted sequences are now [frames, 23, 13]
    lengths = [seq.size(0) for seq in sequences]  # Now seq is [frames, 23, 13]
    
    # Pad sequences to have the same length
    sequences_padded = pad_sequence(sequences, batch_first=True)
    
    labels = torch.tensor(labels, dtype=torch.long)
    lengths = torch.tensor(lengths, dtype=torch.long)

    return sequences_padded, labels, lengths


# Define the model
class RdmClassifier(nn.Module):
    def __init__(self, num_classes, hidden_size):
        super(RdmClassifier, self).__init__()
        # Define a simple CNN architecture
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),  # Assuming RDMs have a single channel
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),  # This will flatten the output of the convolutional layers
        )
        cnn_output_size = self._get_conv_output_size()

        # Define the LSTM layer
        self.lstm = nn.LSTM(cnn_output_size, hidden_size, batch_first=True)
        
        # Define the fully connected layer for classification
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x, lengths):
        batch_size, seq_len, _, _ = x.size()
        # Apply CNN to each RDM in the sequence
        c_out = self.cnn(x.view(batch_size * seq_len, 1, *x.size()[-2:]))
        
        # Reshape for LSTM input
        r_out = c_out.view(batch_size, seq_len, -1)
        
        # Pack the sequence for LSTM
        packed_input = pack_padded_sequence(r_out, lengths, batch_first=True, enforce_sorted=False)
        packed_output, (hidden, _) = self.lstm(packed_input)

        # Use the last hidden state
        out = self.fc(hidden[-1])
        return out
    
    def _get_conv_output_size(self):
        # We can create a dummy input to calculate the output size after the CNN layers
        with torch.no_grad():
            dummy_input = torch.zeros(1, 1, 23, 13)  # Replace with your RDM shape
            dummy_output = self.cnn(dummy_input)
            return dummy_output.size(-1)



In [11]:
# Load the dataset
root_dir = '/Volumes/FourTBLaCie/Yoga_Study_RADAR_1CH_GOUP_BLNC_DOWN'  # Replace with the correct root directory
train_dataset = RdmDataset(root_dir, train=True)
test_dataset = RdmDataset(root_dir, train=False)


# Update your DataLoader to use the new collate_fn
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)


# Hyperparameters
input_size = 3  # Each frame is a 2D Array
hidden_size = 128
num_classes = 3

# Instantiate the model
model = RdmClassifier(num_classes=3, hidden_size=hidden_size)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        for padded_rdm, labels, lengths in train_loader:
            optimizer.zero_grad()
            outputs = model(padded_rdm, lengths)
            loss = criterion(outputs, torch.tensor(labels, dtype=torch.long))
            loss.backward()
            optimizer.step()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')
        

# Complete the evaluate_model function
def evaluate_model(model, test_loader):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for padded_rdm, labels, lengths in test_loader:
            outputs = model(padded_rdm, lengths)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == torch.tensor(labels, dtype=torch.long)).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Train the model
num_epochs = 5  # Define the number of epochs
train_model(model, train_loader, criterion, optimizer, num_epochs)

# Evaluate the model
accuracy = evaluate_model(model, test_loader)
print(f'Accuracy: {accuracy}%')


Is_train? True
['12', '02', '04', '24', '16', '08', '01', '18', '03', '05', '15', '14']
Is_train? False
['02', '03', '10']


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
/var/folders/nh/xjdmv90x37b3xwwbwbm5lp1w0000gn/T/ipykernel_86312/512299123.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = criterion(outputs, torch.tensor(labels, dtype=torch.long))


Epoch [1/5], Loss: 1.1455981731414795
Epoch [2/5], Loss: 0.8852002024650574
Epoch [3/5], Loss: 0.7341263294219971
Epoch [4/5], Loss: 0.04041285067796707
Epoch [5/5], Loss: 0.11543883383274078


/var/folders/nh/xjdmv90x37b3xwwbwbm5lp1w0000gn/T/ipykernel_86312/512299123.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  correct += (predicted == torch.tensor(labels, dtype=torch.long)).sum().item()


Accuracy: 92.1875%
